In [9]:
import os
import socket
import pandas as pd
import geopandas as gpd
import re # regular expressions

def setup_directories():
    # check if we are on the server or local
    nodename = socket.gethostname()
    if nodename == "oMac.local": # local laptop
        root = os.path.expanduser("~/OneDrive - The University of Chicago/guatamala_ag/data")
    else:
        raise Exception("Unknown environment, Please specify the root directory")

    dirs = {
        'root': root,
        'raw': os.path.join(root, "raw"),
        'processed': os.path.join(root, "processed"),
        'fig': os.path.join(root, "../figures")
    }

    for path in dirs.values():
        os.makedirs(path, exist_ok=True)

    return dirs

dir = setup_directories()

In [13]:
df = pd.read_excel(os.path.join(dir['raw'], "Datos de Impacto Productores 2023.xlsx"), 
    sheet_name= 0, skiprows=4)

vars_to_keep = ["id_phone", "id_coordinates_1", "id_coordinates_2", 
                "id_coordinates_3", "id_coordinates_4"]

df = df[vars_to_keep]

# drop rows with missing id_coordinates_1
df = df.dropna(subset=["id_coordinates_1"])

# save the data as a csv
df.to_csv(os.path.join(dir['processed'], "coordinates.csv"), index=False)

def split_coordinates(coord_str):
    # Remove any quotation marks and split on comma or space
    cleaned = coord_str.strip('"')
    parts = re.split(r'[,\s]+', cleaned)
    
    if len(parts) == 2:
        return pd.Series({'latitude': parts[0], 'longitude': parts[1]})
    else:
        return pd.Series({'latitude': '9999', 'longitude': '9999'})

for i in range(1, 5):
    col_name = f'id_coordinates_{i}'
    new_cols = df[col_name].apply(split_coordinates)
    df[f'latitude_{i}'] = new_cols['latitude']
    df[f'longitude_{i}'] = new_cols['longitude']
    # make lat long float
    df[f'latitude_{i}'] = df[f'latitude_{i}'].astype(float)
    df[f'longitude_{i}'] = df[f'longitude_{i}'].astype(float)
    df = df.drop(columns=[col_name])
print(df.head())


    # # make float again
    # data[f'lat_{i}'] = data[f'lat_{i}'].astype(float)
    # data[f'lon_{i}'] = data[f'lon_{i}'].astype(float)

  


/Users/ohouck/envs/guatamala_venv/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell LH147 is marked as a date but the serial value 6729160.0 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


ValueError: could not convert string to float: '13.9553022.'